In [124]:
import uproot
import awkward as ak

# Define your file location
#FILE_LOC = "/data/atlas/data/rho_delta/rho_small.root"
FILE_LOC = "/fast_scratch_1/jbohm/cell_particle_deposit_learning/delta/delta_root_files/delta_full.root"
#FILE_LOC = "/data/atlas/data/rho_delta/delta.root"
#FILE_LOC = "/data/atlas/data/allCellTruthv1/pi0/user.mswiatlo.27153451.OutputStream._000017.root"

events = uproot.open(FILE_LOC+":EventTree")
cell_geo_tree = uproot.open("/data/atlas/data/rho_delta/rho_small.root"+":CellGeo")
queryTree = cell_geo_tree

print(queryTree["cell_geo_ID"].array())
for key in queryTree.keys():
    print(key)





<TBranchElement 'cell_geo_ID' at 0x7f6240ddb310>
cell_geo_ID
cell_geo_sampling
cell_geo_eta
cell_geo_phi
cell_geo_rPerp
cell_geo_deta
cell_geo_dphi
cell_geo_volume
cell_geo_sigma
cell_geo_prevInPhi
cell_geo_nextInPhi
cell_geo_prevInEta
cell_geo_nextInEta
cell_geo_prevInSamp
cell_geo_nextInSamp
cell_geo_prevSubDet
cell_geo_nextSubDet
cell_geo_prevSuperCalo
cell_geo_nextSuperCalo


In [79]:
n_truth_part = events["nTruthPart"].array()


num_particles = 8
first_index = (ak.firsts(ak.where(n_truth_part >= num_particles)))

if ak.is_valid(first_index):
    print(f"Event {first_index} has {n_truth_part[first_index]} truth particles.")
else:
    print(f"No event with {num_particles} or more truth particles was found.")



Event [161730] has [8] truth particles.


In [72]:
import numpy as np
def spherical_to_cartesian(rs, phis, thetas):
    # convert to x, y, z
    x, y, z = [], [], []

    for i, r in enumerate(rs):
        x.append(r*np.cos(phis[i]))
        y.append(r*np.sin(phis[i]))
        z.append( r/np.tan(thetas[i]))

    return np.array(x), np.array(y), np.array(z)

In [120]:
# Define your queries
queries = [
    "cluster_cell_E", 
    "cluster_cell_ID", 
    "cluster_cell_hitsTruthIndex", 
    "cluster_cell_hitsTruthE"
]

# Load the branches for the specified event
items = queryTree.arrays(queries, entry_start=first_index[0], entry_stop=first_index[0] + 1)

# Access the data for each branch
hits_truth_e = items[queries[0]]
hits_truth_id = items[queries[1]]
hits_truth_index = items[queries[2]]
hits_truth_energy = items[queries[3]]

# Print the data and its length for the first event
print(f"Hits Truth Energy ({queries[0]}):", hits_truth_e*1000)
print(f"Length of Hits Truth Energy for the first event", len(hits_truth_e[0][0]))
print()
print(f"Hits Truth IDs ({queries[1]}):", hits_truth_id)
print("Length of Hits Truth IDs for the first event:", len(hits_truth_id[0][0]))
print()
print()
print()
print(f"Hits Truth Index: ({queries[2]})", hits_truth_index)
print("Length of Hits Truth Index for the first event:", len(hits_truth_index[0][0]))
print()
print(f"Hits Truth Energy from Index({queries[3]}):", hits_truth_energy)
print("Length of Hits Truth Energy from Index for the first event:", len(hits_truth_energy[0][0]))
print()
# Count the total number of subarrays and the number of non-empty subarrays
total_subarrays = 0
non_empty_subarrays = 0

print(hits_truth_e[total_subarrays])

# Iterate through the top-level array to count
for event in ak.flatten(hits_truth_energy, axis=2):
    for subarray in event:
        print(total_subarrays,hits_truth_id[0][0][total_subarrays],hits_truth_index[0][0][total_subarrays],subarray,hits_truth_e[0][0][total_subarrays]*1000)
        total_subarrays += 1
        if len(subarray) > 0:
            non_empty_subarrays += 1

print("Total number of subarrays:", total_subarrays)
print("Number of non-empty subarrays:", non_empty_subarrays)


Hits Truth Energy (cluster_cell_E): [[[465, 45, 72, 177, 213, 18, 5.03, ..., 65.2, 48.9, 30, 26, 9.03, 22, 20.9]]]
Length of Hits Truth Energy for the first event 64

Hits Truth IDs (cluster_cell_ID): [[[757123080, 757123078, 757123082, ..., 757126662, 757126666, 757127174]]]
Length of Hits Truth IDs for the first event: 64



Hits Truth Index: (cluster_cell_hitsTruthIndex) [[[[6, 5], [6, 5], [5], [6, 5], [6, 5], [5], ..., [], [], [], [], [], []]]]
Length of Hits Truth Index for the first event: 83

Hits Truth Energy from Index(cluster_cell_hitsTruthE): [[[[7.28, 463], [3.67, 17], [84.1], [5.87, 129], [...], ..., [], [], [], []]]]
Length of Hits Truth Energy from Index for the first event: 83

[[0.465, 0.045, 0.072, 0.177, 0.213, ..., 0.03, 0.026, 0.00903, 0.022, 0.0209]]
0 757123080 [6, 5] [7.28, 463] 465.2174711227417
1 757123078 [6, 5] [3.67, 17] 45.012135058641434
2 757123082 [5] [84.1] 71.982242166996
3 757122568 [6, 5] [5.87, 129] 177.20970511436462
4 757123592 [6, 5] [6.43, 233]

IndexError: cannot slice NumpyArray (of length 64) with 64: index 64 is out of bounds for axis 0 with size 64

This error occurred while attempting to slice

    <Array [757123080, 757123078, ..., 757126666, 757127174] type='64 * uint64'>

with

    64